In [1]:
from google.colab import auth

# Aunthenticate to Google Colab
auth.authenticate_user()

#Configure Google Cloud
from tools import load_db_cfg
dbcfg=load_db_cfg('cfg.json')
!gcloud config set project {dbcfg['project_id']}

'''
  Enable Google SQL Admin API Services
'''
# Enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

Updated property [core/project].


In [ ]:
!pip install cloud-sql-python-connector

In [ ]:
'''
  Drop DB if exists and create it afresh
'''
from tools import load_db_cfg
dbcfg=load_db_cfg('cfg.json')
db_name=dbcfg['name']
instance_name = dbcfg['instance_name']
#!gcloud sql databases delete {db_name} --instance={instance_name}

print(f'''Database '{db_name}' deleted successfully !!''')


In [ ]:
!gcloud sql databases create {db_name} --instance={instance_name}
print(f'''Database '{db_name}' created successfully !!''')


In [ ]:
# Install PyMySQL python module
!pip install pymysql

In [4]:
'''
  Connect to Cloud SQL DB to create the schema for 'youtube_info' database
'''
from tools import load_db_cfg
from ytdbconnect import YtDbConnector
dbcfg=load_db_cfg('cfg.json')
pool = YtDbConnector(dbcfg)


In [ ]:
'''
  Create the required tables for 'youtube_info' database
'''
import sqlalchemy
with pool.connect() as db:
  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS channel (
        id VARCHAR(255) NOT NULL,
        name VARCHAR(255) NOT NULL,
        type VARCHAR(255) NOT NULL,
        views BIGINT NOT NULL,
        description TEXT,
        status VARCHAR(255),
        playlist_id VARCHAR(255),
        INDEX idx1 (playlist_id),
        PRIMARY KEY (id));''')
  )

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS playlist (
        id VARCHAR(255) NOT NULL,
        name VARCHAR(255) NOT NULL,
        channel_id VARCHAR(255) NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (channel_id) REFERENCES channel(id) ON DELETE CASCADE);''')
  )

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS video (
          id VARCHAR(255) NOT NULL,
          playlist_id VARCHAR(255) NOT NULL,
          channel_id VARCHAR(255) NOT NULL,
          name VARCHAR(255) NOT NULL,
          description TEXT NOT NULL,
          published_date DATETIME NOT NULL,
          view_count BIGINT NOT NULL DEFAULT 0,
          like_count BIGINT NOT NULL DEFAULT 0,
          dislike_count BIGINT NOT NULL DEFAULT 0,
          favorite_count BIGINT NOT NULL DEFAULT 0,
          comment_count BIGINT NOT NULL DEFAULT 0,
          duration BIGINT NOT NULL DEFAULT 0,
          thumbnail VARCHAR(255),
          caption_status VARCHAR(255) NOT NULL,
          PRIMARY KEY (id),
          FOREIGN KEY (channel_id) REFERENCES channel(id) ON DELETE CASCADE);''')
  )

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS comment (
        id VARCHAR(255) NOT NULL,
        video_id VARCHAR(255) NOT NULL,
        comment_text TEXT NOT NULL,
        author  VARCHAR(255) NOT NULL,
        published_date DATETIME NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (video_id) REFERENCES video(id) ON DELETE CASCADE); ''')
  )

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS thumbnails_in_video (
          video_id VARCHAR(255) NOT NULL,
          type VARCHAR(255) NOT NULL,
          url VARCHAR(255) NOT NULL,
          FOREIGN KEY (video_id) REFERENCES video(id) ON DELETE CASCADE);''')
  )

  # Commit the above changes to DB
  db.commit()
  print( f'''Successfully created the tables for '{db_name}' database !! You are good to go !!''')
  db.close()


Successfully connected to 'youtube' database!! 
Successfully created the tables for 'youtube' database !! You are good to go !!
